In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


In [2]:
data = pd.read_csv("data.csv")

data.head()

,date,competition_stage_euro,home_team_euro,away_team_euro,hour_euro,equipe_fr_euro,top_equipe_euro,note_euro,competition_stage_cdm,home_team_cdm,...,moyenne_matin,grande_matin,géante_matin,plaque_matin,nombre_pizza_matin,nombre_clients_matin,weekday,month,day_month,year
0,2018-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Monday,January,8,2018
1,2018-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,24.0,13.0,1.0,0.0,41.0,17.0,Tuesday,January,9,2018
2,2018-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,24.0,23.0,2.0,0.0,65.0,34.0,Wednesday,January,10,2018
3,2018-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,13.0,2.0,9.0,32.0,26.0,Thursday,January,11,2018
4,2018-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.0,19.0,0.0,0.0,72.0,36.0,Friday,January,12,2018


In [3]:
print(data.columns)

Index(['date', 'competition_stage_euro', 'home_team_euro', 'away_team_euro',
       'hour_euro', 'equipe_fr_euro', 'top_equipe_euro', 'note_euro',
       'competition_stage_cdm', 'home_team_cdm', 'away_team_cdm', 'hour_cdm',
       'equipe_fr_cdm', 'top_equipe_cdm', 'note_cdm', 'competition_stage_ldc',
       'home_team_ldc', 'away_team_ldc', 'hour_ldc', 'club_fr_ldc',
       'top_club_ldc', 'note_ldc', 'heure_rugby_fr', 'num_journee_rugby_6n',
       'nbr_v_av_m_fr_6n', 'Tour_rugby_fr', 'competition_rugby_fr',
       'heure_STD', 'compétition_STD', 'tour_STD', 'temperature', 'jour_spe',
       'nom_vacances', 'confinement', 'heure_couvre_feu', 'petite_jour',
       'moyenne_jour', 'grande_jour', 'géante_jour', 'plaque_jour',
       'nombre_pizza_jour', 'Total_jour', 'nombre_clients_jour', 'petite_soir',
       'moyenne_soir', 'grande_soir', 'géante_soir', 'plaque_soir',
       'nombre_pizza_soir', 'nombre_clients_soir', 'petite_matin',
       'moyenne_matin', 'grande_matin', 'géante_m

In [4]:
print(data['confinement'].unique())

[nan 'Confinement 1' 'Confinement 2' 'Confinement 3']


In [5]:
data = data[['equipe_fr_euro', 'equipe_fr_cdm', 'heure_rugby_fr', 'heure_STD', 'nom_vacances', 'confinement',
            'nombre_pizza_soir', 'nombre_pizza_matin', 'weekday', 'month']]
data.head()

,equipe_fr_euro,equipe_fr_cdm,heure_rugby_fr,heure_STD,nom_vacances,confinement,nombre_pizza_soir,nombre_pizza_matin,weekday,month
0,NaN,NaN,NaN,NaN,NaN,NaN,104.0,NaN,Monday,January
1,NaN,NaN,NaN,NaN,NaN,NaN,148.0,41.0,Tuesday,January
2,NaN,NaN,NaN,NaN,NaN,NaN,203.0,65.0,Wednesday,January
3,NaN,NaN,NaN,NaN,NaN,NaN,188.0,32.0,Thursday,January
4,NaN,NaN,NaN,NaN,NaN,NaN,369.0,72.0,Friday,January


In [6]:
data['match_fr_foot'] = data.apply(lambda row: True if 1 in [row['equipe_fr_cdm'], row['equipe_fr_euro']] else False, axis=1)

data['match_fr_rugby'] = data['heure_rugby_fr'].notna()

data['match_stade'] = data['heure_STD'].notna()

data['confinement'] = data['confinement'].notna()

data['vacance'] = data['nom_vacances'].notna()

In [7]:
data = data[['confinement', 'weekday', 'month', 'match_fr_foot', 'match_fr_rugby',
           'match_stade', 'vacance', 'nombre_pizza_soir']]

data = data.dropna()
print(data[['weekday', 'month']].isna().sum())

data.shape

weekday    0
month      0
dtype: int64


(1939, 8)

In [8]:
encoder_weekday = OneHotEncoder()
encoder_month = OneHotEncoder()

encoded_weekday = encoder_weekday.fit_transform(data[['weekday']])
encoded_month = encoder_month.fit_transform(data[['month']])

encoded_weekday_df = pd.DataFrame(encoded_weekday.toarray(), columns=encoder_weekday.get_feature_names_out(['weekday']))
encoded_month_df = pd.DataFrame(encoded_month.toarray(), columns=encoder_month.get_feature_names_out(['month']))

encoded_weekday_df.reset_index(drop=True, inplace=True)
encoded_month_df.reset_index(drop=True, inplace=True)
data.reset_index(drop=True, inplace=True)

data = pd.concat([data.drop(['weekday', 'month'], axis=1), encoded_weekday_df, encoded_month_df], axis=1)

In [9]:
target = 'nombre_pizza_soir'

cols = list(data.columns)

cols.remove(target)

cols.append(target)

data_final = data[cols]

data_final.head()

,confinement,match_fr_foot,match_fr_rugby,match_stade,vacance,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,...,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September,nombre_pizza_soir
0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0
1,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,148.0
2,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,203.0
3,False,False,False,False,False,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,188.0
4,False,False,False,False,False,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,369.0


In [10]:
data_final.to_csv("data-v1.csv")